# Data Assessment 2 — OTT Revenue Cohorts by Country

This notebook cleans, links, and analyzes subscription revenue to evaluate first-purchase cohorts by country:
- Cohort A: Non-Offer First Joiners (no coupon on first purchase)
- Cohort B: Offer First Joiners (coupon on first purchase)

Outputs include an advanced, management-ready table and interactive charts (Plotly).

Objectives:
- Build a clean, analyzable dataset across customers, countries, revenue, subscriptions, and plans.
- Segment first purchases into Non-Offer vs Offer cohorts.
- Produce advanced country × cohort metrics for decisions.
- Visualize results with interactive tables and charts.
- Summarize data-backed proposals to grow revenue.

How to Navigate:
- Load & Clean Data: sources and data quality rules.
- Cohorts & Outcomes: definitions and per-customer logic.
- Country × Cohort Metrics: management table.
- Visuals: LTV vs Repeat; Joiners by month; Plan mix.


## Analysis Overview

Purpose
- Diagnose country-level purchase behavior split by first-purchase cohorts to explain flat targets and identify where lifetime value concentrates.

Inputs
- Customers, Countries, Revenue (orders), Subscriptions, Plans from `data/`.

Preparation
- Standardize columns, parse mixed dates, coerce amounts, drop invalid payments, deduplicate by order/transaction, normalize coupon codes, and join subscriptions → revenue → customers → countries → plans.

Cohorts
- Non-Offer First Joiners: first purchase without coupon.
- Offer First Joiners: first purchase with coupon.

Repeat Logic
- A repeat is any purchase strictly after the first purchase date; compute first repeat date and days-to-repeat.

Metrics (Country × Cohort)
- New customers, first revenue, first ARPU, repeat customers, repeat rate, repeat revenue, average LTV (first + repeats), median days to first repeat, top first-purchase plan, and country offer share.

Visuals
- Interactive management table, LTV vs repeat rate (bubble by scale), monthly first joiners, and first-plan mix.

How to Use
- Run all cells after installing requirements; use interactive controls (hover/zoom/legend) to explore and filter insights.


In [42]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
pd.set_option('display.max_columns', 100)
base = Path('data')
assert base.exists(), 'Expected data/ folder in project root'

## Load and Clean Data

Data sources (from `data/`):
- `Revenue - customers.csv`: customer id, email, name, country_id.
- `Revenue - countries.csv`: country id → country name.
- `Revenue - subscription.csv`: subscription id, customer_id, order_id, plan_id.
- `Revenue - revenue.csv`: order-level payments (date, amount, plan_id, method, coupon).
- `Revenue - plan.csv`: plan id → plan name.

Cleaning and normalization applied:
- Standardized column names; trimmed whitespace and quotes.
- Parsed mixed `payment_date` formats into valid datetimes.
- Coerced `amount` to numeric, dropping invalid rows.
- Deduplicated payments by earliest valid `order_id` + `transaction_id`.
- Treated empty/`#N/A` coupons as non-offer.
- Joined subscriptions → revenue → customers → countries → plans.


In [43]:
countries = pd.read_csv(base / 'Revenue - countries.csv')
customers = pd.read_csv(base / 'Revenue - customers.csv', dtype=str)
plans = pd.read_csv(base / 'Revenue - plan.csv')
revenue = pd.read_csv(base / 'Revenue - revenue.csv', dtype=str)
subs = pd.read_csv(base / 'Revenue - subscription.csv', dtype=str)

def norm_cols(df):
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    return df
countries = norm_cols(countries)
customers = norm_cols(customers)
plans = norm_cols(plans)
revenue = norm_cols(revenue)
subs = norm_cols(subs)

customers = customers[[c for c in ['id','email','name','country_id'] if c in customers.columns]]
customers['id'] = customers['id'].astype(str)
customers['country_id'] = customers['country_id'].astype(str)
countries['id'] = countries['id'].astype(str)

if 'id' in plans.columns and 'plan_name' in plans.columns:
    plan_map = plans.rename(columns={'id':'plan_id'})[['plan_id','plan_name']]
elif 'Id' in plans.columns:
    plan_map = plans.rename(columns={'Id':'plan_id'})[['plan_id','plan_name']]
else:
    plan_map = plans
plan_map['plan_id'] = plan_map['plan_id'].astype(str)

rev = revenue.copy()
for col in ['payment_date','amount','coupon_code','order_id','transaction_id','plan_id','payment_method']:
    if col in rev.columns:
        rev[col] = rev[col].astype(str).str.strip().str.strip('"')
rev['coupon_code'] = rev['coupon_code'].replace({'#N/A': np.nan, 'nan': np.nan, 'None': np.nan, '': np.nan})
rev['amount_clean'] = rev['amount'].str.replace(r'[^0-9\.-]','', regex=True)
rev['amount'] = pd.to_numeric(rev['amount_clean'], errors='coerce')
rev['payment_date'] = pd.to_datetime(rev['payment_date'], errors='coerce')
rev = rev.dropna(subset=['payment_date','amount'])
rev['plan_id'] = rev['plan_id'].astype(str)
dedup_keys = ['order_id','transaction_id'] if 'transaction_id' in rev.columns else ['order_id']
rev = rev.sort_values('payment_date').drop_duplicates(subset=dedup_keys, keep='first')

for col in ['order_id','customer_id','plan_id']:
    subs[col] = subs[col].astype(str).str.strip()
subs = subs.drop_duplicates(subset=['id'])

orders = subs.merge(rev, on=['order_id','plan_id'], how='inner', validate='m:1')
orders = orders[['customer_id','order_id','plan_id','payment_date','amount','payment_method','coupon_code']]
orders = orders.merge(plan_map, on='plan_id', how='left')
orders = orders.merge(customers.rename(columns={'id':'customer_id'}), on='customer_id', how='left')
orders = orders.merge(countries.rename(columns={'id':'country_id','country_name':'country'}), on='country_id', how='left')
orders = orders.sort_values(['customer_id','payment_date','order_id']).reset_index(drop=True)

## Cohorts and Customer-Level Outcomes

Cohort definitions:
- Non-Offer First Joiners: first purchase without a coupon.
- Offer First Joiners: first purchase with a coupon.

Per-customer outcomes:
- First metrics: date, plan, amount.
- Repeat: any purchase strictly after the first date.
- Days to first repeat: first repeat date − first purchase date.
- LTV in window: first amount + repeat revenue.


In [44]:
firsts = orders.groupby('customer_id', as_index=False).first()
firsts = firsts.rename(columns={
    'order_id':'first_order_id',
    'payment_date':'first_date',
    'amount':'first_amount',
    'coupon_code':'first_coupon',
    'plan_id':'first_plan_id',
    'plan_name':'first_plan_name'
})
firsts['cohort'] = np.where(firsts['first_coupon'].notna() & (firsts['first_coupon'].astype(str).str.len()>0), 'Offer First Joiners','Non-Offer First Joiners')

rests = orders.merge(firsts[['customer_id','first_date']], on='customer_id', how='left')
repeats = rests[rests['payment_date'] > rests['first_date']].copy()
first_repeat = repeats.groupby('customer_id')['payment_date'].min().reset_index(name='first_repeat_date')
repeat_flags = repeats.groupby('customer_id').size().reset_index(name='repurchase_count')
repeat_flags['repeat_flag'] = 1
cust_repeat = (
    firsts[['customer_id','country','cohort','first_amount','first_date']]
    .merge(repeat_flags[['customer_id','repeat_flag']], on='customer_id', how='left')
    .merge(repeats.groupby('customer_id').agg(repeat_revenue=('amount','sum'), repeat_orders=('order_id','nunique')).reset_index(), on='customer_id', how='left')
    .merge(first_repeat, on='customer_id', how='left')
)
cust_repeat['repeat_flag'] = cust_repeat['repeat_flag'].fillna(0)
cust_repeat['repeat_revenue'] = cust_repeat['repeat_revenue'].fillna(0)
cust_repeat['days_to_repeat'] = (cust_repeat['first_repeat_date'] - cust_repeat['first_date']).dt.days

## Country × Cohort Advanced Metrics

Metric dictionary:
- New customers: unique customers whose first purchase falls in the data window.
- First revenue / ARPU: sum/average of first purchase amounts.
- Repeat customers / rate: customers with >0 repeat purchases and the share.
- Repeat revenue: revenue from purchases after first date.
- LTV avg: mean of per-customer total (first + repeats).
- Median days to repeat: among repeaters.
- First plan top: most common plan at first purchase.
- Offer share: share of first joiners using a coupon (country level).


In [45]:
g1 = firsts.groupby(['country','cohort'], dropna=False).agg(
    new_customers=('customer_id','nunique'),
    first_revenue=('first_amount','sum'),
    first_arpu=('first_amount','mean'),
    first_plan_top=('first_plan_name', lambda x: x.value_counts(dropna=False).index[0] if len(x)>0 else np.nan)
).reset_index()

g2 = cust_repeat.groupby(['country','cohort']).agg(
    repeat_customers=('repeat_flag','sum'),
    repeat_rate=('repeat_flag','mean'),
    repeat_revenue=('repeat_revenue','sum'),
    orders_per_customer=('repeat_orders','mean'),
    median_days_to_repeat=('days_to_repeat','median')
).reset_index()

total_per_cust = firsts[['customer_id','country','cohort','first_amount']].merge(
    cust_repeat[['customer_id','repeat_revenue']], on='customer_id', how='left'
)
total_per_cust['total_rev'] = total_per_cust['first_amount'] + total_per_cust['repeat_revenue'].fillna(0)
ltv = total_per_cust.groupby(['country','cohort']).agg(ltv_avg=('total_rev','mean')).reset_index()

summary = (
    g1.merge(g2, on=['country','cohort'], how='left').merge(ltv, on=['country','cohort'], how='left')
)
summary[['repeat_customers','repeat_rate','repeat_revenue','orders_per_customer','median_days_to_repeat']] = \
    summary[['repeat_customers','repeat_rate','repeat_revenue','orders_per_customer','median_days_to_repeat']].fillna(0)

country_totals = summary.groupby('country')['new_customers'].sum().reset_index(name='new_total')
offer = summary[summary['cohort']=='Offer First Joiners'][['country','new_customers']].rename(columns={'new_customers':'offer_new'})
offer_share = country_totals.merge(offer, on='country', how='left').fillna({'offer_new':0})
offer_share['offer_share'] = offer_share['offer_new']/offer_share['new_total']
summary = summary.merge(offer_share[['country','offer_share']], on='country', how='left')
summary = summary.sort_values(['country','cohort']).reset_index(drop=True)

## Interactive Table (Management View)

How to read:
- Sort by LTV Avg to prioritize high-value markets.
- Compare Repeat Rate across cohorts to assess stickiness.
- Use Offer Share to spot promo dependence and risk.
- First Plan Top guides plan-led creative and upsell timing.


In [46]:
display_cols = [
    'country','cohort','new_customers','first_revenue','first_arpu',
    'repeat_customers','repeat_rate','repeat_revenue','ltv_avg',
    'median_days_to_repeat','first_plan_top','offer_share'
]
tbl = summary[display_cols].copy()
# Keep rates as proportions and format via Plotly for readability
tbl = tbl.sort_values(['ltv_avg','repeat_rate'], ascending=[False, False])
import plotly.graph_objects as go
row_h, header_h = 26, 34
col_align = ['left','left','right','right','right','right','right','right','right','right','left','right']
col_formats = [None, None, ',d', ',.2f', ',.2f', ',d', '.2%', ',.2f', ',.2f', ',d', None, '.2%']
col_widths = [220, 190, 110, 140, 120, 140, 120, 150, 110, 190, 140, 120]
fig_tbl = go.Figure(data=[go.Table(
    columnorder=list(range(1, len(display_cols)+1)),
    columnwidth=col_widths,
    header=dict(
        values=[c.replace('_',' ').title() for c in tbl.columns],
        fill_color='#1f77b4',
        font=dict(color='white', size=13),
        align=['left','left','right','right','right','right','right','right','right','right','left','right'],
        height=header_h
    ),
    cells=dict(
        values=[tbl[c] for c in tbl.columns],
        align=col_align,
        format=col_formats,
        height=row_h,
        fill=dict(color='#F8FAFF'),
        font=dict(size=12),
        line_color='#E5ECF6'
    )
)])
fig_tbl.update_layout(
    title='Country × Cohort Advanced Metrics',
    height=900,
    margin=dict(l=0, r=0, t=60, b=0)
)
fig_tbl.show()

## LTV vs Repeat Rate (by Cohort)

Interpretation:
- Upper-right countries are high LTV and high repeat — growth priority.
- Large bubbles = many first joiners: balance value vs scale.
- Compare colors to see cohort differences by market.


In [47]:
fig = px.scatter(
    summary, x='repeat_rate', y='ltv_avg', color='cohort', size='new_customers',
    hover_data=['country','first_arpu','repeat_revenue','offer_share'],
    labels={'repeat_rate':'Repeat Rate','ltv_avg':'Average LTV'},
    title='Country LTV vs Repeat Rate — bubble size = new customers'
)
fig.update_layout(xaxis_tickformat='.0%', legend_title_text='Cohort')
fig.show()

## First Joiners by Month and Cohort

Interpretation:
- Detect acquisition seasonality and campaign lift.
- Watch for cohort mix shifts that signal over-reliance on promos.


In [48]:
firsts['first_month'] = firsts['first_date'].values.astype('datetime64[M]')
by_month = firsts.groupby(['first_month','cohort']).size().reset_index(name='new_customers')
fig = px.line(by_month, x='first_month', y='new_customers', color='cohort', markers=True,
              title='First Joiners by Month and Cohort')
fig.update_layout(xaxis_title='Month', yaxis_title='New Customers')
fig.show()

## Plan Mix at First Purchase

Interpretation:
- Use plan mix to set upgrade offers (e.g., Monthly → 3/6/12).
- Differences by cohort inform targeted messaging.


In [49]:
plan_mix = firsts.groupby(['cohort','first_plan_name']).size().reset_index(name='count')
plan_mix['share'] = plan_mix.groupby('cohort')['count'].transform(lambda s: s/s.sum())
fig = px.bar(plan_mix, x='first_plan_name', y='share', color='cohort', barmode='group',
             text_auto='.0%', title='First-Purchase Plan Mix by Cohort')
fig.update_layout(xaxis_title='Plan', yaxis_title='Share of First Purchases', yaxis_tickformat='.0%')
fig.show()